# Final version

In [ ]:
def date_sorter():
    import re
    import numpy as np
    from datetime import datetime

    ######################################## GET DATES ########################################
    def matcher(x):
        # For later use
        month_mapper = {
            1 : ("Jan", "Jan.", "January", "January."),
            2 : ("Feb", "Feb.", "February", "February."),
            3 : ("Mar", "Mar.", "March", "March."),
            4 : ("Apr", "Apr.", "April", "April."),
            5 : ("May", "May.", "May", "May."),
            6 : ("Jun", "Jun.", "June", "June."),
            7 : ("Jul", "Jul.", "July", "July."),
            8 : ("Aug", "Aug.", "August", "August."),
            9 : ("Sep", "Sep.", "September", "September."),
            10 : ("Oct", "Oct.", "October", "October."),
            11 : ("Nov", "Nov.", "November", "November."),
            12 : ("Dec", "Dec.", "December", "December."),
        }
        ############################## MATCHER 1 ##############################
        try:
            ###### MATCH: 04/20/2009; 04/20/09; 4/20/09; 4/3/09
            match = re.findall(r"[\d]{1,2}[/\-][\d]{1,2}[/\-][\d]{2,4}", x)[0]
            ###### PROCESSING
            if "/" in match:
                items = match.split("/")
            if "-" in match:
                items = match.split("-")

            if len(items[0]) == 1:
                items[0] = "0" + items[0]

            if len(items[1]) == 1:
                items[1] = "0" + items[1]

            if len(items[2]) == 2:
                items[2] = "19" + items[-1]

            processed_match = "/".join(items)
            return processed_match
        except:        
            ############################## MATCHER 2 ##############################
            try:
                ###### MATCH: Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009
                match = re.findall(r"[^\W\d_]{3,}[\-\s\.]{1,2}[\d]{1,2}[\-\s\,]{1,2}[\d]{4}", x)[0]
                ###### PROCESSING
                items = match.replace(",", "").split(" ")

                for i, k in month_mapper.items():
                    if items[0] in k:
                        items[0] = str(i)

                if len(items[0]) == 1:
                    items[0] = "0" + items[0]

                if len(items[1]) == 1:
                    items[1] = "0" + items[1]

                processed_match = "/".join(items)
                return processed_match
            except:
                ############################## MATCHER 3 ##############################
                try:
                    ###### MATCH: 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
                    match = re.findall(r"[\d]{1,2}\s[^\W\d_]{3,}[\-\s\.]{1,2}[\d]{4}", x)[0]
                    ###### PROCESSING
                    items = match.replace(",", "").split(" ")

                    for i, k in month_mapper.items():
                        if items[1] in k:
                            items[1] = str(i)

                    if len(items[0]) == 1:
                        items[0] = "0" + items[0]

                    if len(items[1]) == 1:
                        items[1] = "0" + items[1]

                    day = items[0]
                    month = items[1]

                    items[1] = day
                    items[0] = month

                    processed_match = "/".join(items)
                    return processed_match
                except:
                    ############################## MATCHER 4 ##############################
                    try:
                        ###### MATCH: Feb 2009; Sep 2009; Oct 2010
                        match = re.findall(r"[^\W\d_]{3,}\s[\d]{4}", x)[0]
                        ###### PROCESSING
                        items = match.replace(",", "").split(" ")

                        for i, k in month_mapper.items():
                            if items[0] in k:
                                items[0] = str(i)

                        if len(items[0]) == 1:
                            items[0] = "0" + items[0]

                        items.insert(1, "01")

                        processed_match = "/".join(items)
                        return processed_match
                    except:
                        ############################## MATCHER 5 ##############################
                        try:
                            ###### MATCH: 6/2008; 12/2009
                            match = re.findall(r"[\d]{1,2}/[\d]{4}", x)[0]
                            ###### PROCESSING
                            items = match.replace(",", "").split("/")

                            if len(items[0]) == 1:
                                items[0] = "0" + items[0]

                            items.insert(1, "01")

                            processed_match = "/".join(items)
                            return processed_match
                        except:
                            ############################## MATCHER 6 ##############################
                            try:
                                ###### MATCH: 2009; 2010
                                match = re.findall(r"[\d]{4}", x)[0]
                                ###### PROCESSING
                                items = match.replace(",", "").split("/")

                                items.insert(0, "01")
                                items.insert(0, "01")

                                processed_match = "/".join(items)
                                return processed_match
                            except:
                                return np.nan

    ######################################## PROCESSING ########################################
    def to_time(x):
        try:
            return datetime.strptime(x, "%m/%d/%Y")
        except:
            return np.nan

    # 1) GET DATES
    df2 = df.apply(lambda x: matcher(x))

    # 2) CLEAN OUTLIERS
    df2[235] = "10/01/2015"
    df2[245] = "11/01/1990"
    df2[256] = "08/01/1988"
    df2[281] = "08/01/2004"
    df2[283] = "02/01/1977"
    df2[285] = "09/01/1977"
    df2[298] = "01/01/1993"
    df2[313] = "12/01/1978"
    df2[333] = "11/01/1997"
    df2[459] = "01/01/1998"
    df2[460] = "01/01/2012"
    df2[461] = "01/01/1991"
    df2[467] = "01/01/2011"
    df2[468] = "01/01/1997"
    df2[479] = "01/01/1996"
    df2[487] = "01/01/1992"
    #df2[488] = "01/01/1977"

    # 3) TRANSFORM INTO DATE FORMAT
    df2 = df2.apply(lambda x: to_time(x))

    # 4) SORTING DATA
    df2 = df2.reset_index()
    return df2.sort_values(by = 0).index